T5 Flan
----
This notebook follows this guide: https://huggingface.co/docs/transformers/tasks/summarization

# Setup

In [1]:
! pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.9/769.9 kB 21.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 19.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, create_optimizer, AdamWeightDecay, pipeline
from transformers.keras_callbacks import KerasMetricCallback
import evaluate
import numpy as np

2023-07-28 12:02:12.141392: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 12:02:12.148999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 12:02:12.259425: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 12:02:12.261205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 12:02:13.095440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Defining the model to use.

In [2]:
checkpoint = 'google/flan-t5-small'

# Data

In [3]:
billsum = load_dataset('billsum', split='ca_test').train_test_split(test_size=0.2)

# HF data objects can be indexed EITHER by obs or key: the former returns a dict, the latter a list
print(billsum['train'][0])
print(billsum['train']['summary'][:5])

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 21655.1 is added to the Vehicle Code, to read:\n21655.1.\n(a) A person shall not operate a motor vehicle on a portion of a highway that has been designated for the exclusive use of public transit buses, except in compliance with the directions of a peace officer or official traffic control device.\n(b) This section does not apply to a driver who is required to enter a lane designated for the exclusive use of public transit buses in order to make a right turn or a left turn in a location where there is no left-turn lane for motorists, or who is entering into or exiting from a highway, unless there are signs prohibiting turns across the lane or the lane is delineated by a physical separation, including, but not limited to, a curb, fence, landscaping, or other barrier.\n(c) A public transit agency, with the agreement of the agency with jurisdiction over the highway, shall place and maintain, or c

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
def preprocess_function(data):
    prefix = "summarize: "
    inputs = [prefix + text for text in data['text']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    labels = tokenizer(text_target=data['summary'], max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [6]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=checkpoint,
    return_tensors='tf'
)

Map: 100%|██████████| 248/248 [00:00<00:00, 449.39 examples/s]


# Defining Model and Metrics

In [7]:
rouge = evaluate.load('rouge')
def compute_metrics(eval_pred, evalutor=rouge, tokenizer=tokenizer):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # not sure what this is doing
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = evalutor.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


Note that HF models have built-in loss functions, so one does not need to be specified when compiling.

In [8]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.compile(optimizer=optimizer)

train_set = model.prepare_tf_dataset(
    tokenized_billsum['train'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)
test_set = model.prepare_tf_dataset(
    tokenized_billsum['test'],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=test_set)

model.fit(
    x=train_set,
    validation_data=test_set,
    epochs=3,
    callbacks=[metric_callback]
)

/home/mark/.local/share/virtualenvs/QualAbstracts-R4Ruw4b-/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the 

Epoch 1/3


: 

: 